In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from config import g_key

In [2]:
#Load in csv file from part 1 and store in dataframe
csv_file = "../WeatherPy/weather_dataframe.csv"

city_df = pd.read_csv(csv_file)

In [3]:
city_df = city_df.drop('Unnamed: 0', axis=1)

## Humidity Heatmap

In [4]:
#Create heatmap that displays the humidity for every city
#Use lat and lang as location and humidity as weight 

gmaps.configure(g_key)

#Set locations for heat map
locations = city_df[["Lat","Lng"]]

#Set humidity variable for heatmap weight
humidity = city_df['Humidity']

In [5]:
#Load heat map figure
fig = gmaps.figure()

#Set heat layer as locations and humidity as weight
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

#Add heat layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## New Dataframe with ideal weather criteria

In [6]:
#Create new dataframe displaying ideal weather conditions

#Set max temp as between 70 and 90
max_temp = city_df.loc[(city_df['Max Temp'] >= 70) & (city_df['Max Temp'] <= 90)]

#Set wind speed as less than 15
wind_speed = max_temp.loc[max_temp['Wind Speed'] < 15]

#Set cloudiness as less than 10
cloudiness = wind_speed.loc[wind_speed['Cloudiness'] < 10]

In [21]:
#Create new hotel DF with only locations that meet ideal weather conditions
hotel_df = cloudiness.copy()
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,rikitea,-23.12,-134.97,73.60,79,0,14.65,PF,1603232105
58,mitsamiouli,-11.38,43.28,78.85,72,0,1.77,KM,1603232112
70,quelimane,-17.88,36.89,77.40,74,0,7.90,MZ,1603232115
72,gat,31.61,34.76,75.00,100,0,3.65,IL,1603231888
78,bandrele,-12.91,45.19,77.00,83,0,3.36,YT,1603232117


## Find hotels within 5000 meters of ideal weather locations

In [22]:
#Create new empty column for storing hotel names
hotel_df['Hotel'] = ""

In [23]:
#Find first hotel for each city within 5000 meters using google places api

#Set params to search for hotels within 5000 meters
params = {"key": g_key,
         "type":"lodging",
         "radius": 5000}

#Create for loop using itterows to find the nearest hotel for each location
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel'] = response['results'][0]['name']
    except (IndexError):
        print("Missing")


Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing


In [27]:
#Display hotel df with hotel names now added
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
21,rikitea,-23.12,-134.97,73.60,79,0,14.65,PF,1603232105,Pension Maro'i
58,mitsamiouli,-11.38,43.28,78.85,72,0,1.77,KM,1603232112,Foyer ADM
70,quelimane,-17.88,36.89,77.40,74,0,7.90,MZ,1603232115,Hotel Flamingo
72,gat,31.61,34.76,75.00,100,0,3.65,IL,1603231888,OlusHome
78,bandrele,-12.91,45.19,77.00,83,0,3.36,YT,1603232117,Les Baobabs
85,kununurra,-15.77,128.73,78.80,94,0,3.36,AU,1603232118,Hotel Kununurra
111,menongue,-14.66,17.69,71.22,21,0,5.88,AO,1603232124,Ritz Lauca
115,hilo,19.73,-155.09,84.20,58,1,12.75,US,1603232125,Hilo Hawaiian Hotel
116,san jose,13.30,-4.90,85.57,27,0,2.48,ML,1603231961,Hôtel Teriya
138,atar,20.52,-13.05,89.01,23,0,7.38,MR,1603232130,Oumou Elghoura


## Add hotel markers to heatmap

In [28]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure with hotel markers
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))